### 1. 导入相关的包

In [1]:
import evaluate
import numpy as np
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

### 2. 加载数据集

In [ ]:
dataset = load_dataset("peoples_daily_ner", cache_dir="./data")

Generating train split:   0%|          | 0/20865 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2319 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4637 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [5]:
dataset["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [6]:
# 查看各个字段的属性
dataset["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [7]:
# 取出标签列表 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
label_lst = dataset["train"].features["ner_tags"].feature.names
label_lst

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

### 3. 数据预处理

In [9]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base", cache_dir="./tokenizer")

In [ ]:
# example 1:
tokenizer(dataset["train"][0]["tokens"])

{'input_ids': [[101, 3862, 102], [101, 7157, 102], [101, 3683, 102], [101, 6612, 102], [101, 1765, 102], [101, 4157, 102], [101, 1762, 102], [101, 1336, 102], [101, 7305, 102], [101, 680, 102], [101, 7032, 102], [101, 7305, 102], [101, 722, 102], [101, 7313, 102], [101, 4638, 102], [101, 3862, 102], [101, 1818, 102], [101, 511, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [11]:
tokenizer.decode([101, 102])

'[CLS] [SEP]'

In [ ]:
# 存在一个问题: tokenizer 把每个汉字都视为一个句子，加上了 [CLS] 和 [SEP] 特殊标记
# 我们希望只在完整的句子前后加上一次 [CLS] 和 [SEP] 标记即可，通过指定 is_split_into_words=True 来实现
tokenizer(dataset["train"][0]["tokens"], is_split_into_words=True)

# 结论: 对于已经分词好的文本，需要指定 is_split_into_words=True

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
# example 2:
ret = tokenizer("it is interesting")
ret

{'input_ids': [101, 8233, 8310, 10673, 12865, 12921, 8181, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# it is interesting 三个单词却被分成了 5 个 token (8233, 8310, 10673, 12865, 12921, 8181)
# 英文数据集常常会出现这样的情况，那么如何保证 token 与 word 一一对应
ret.word_ids()  

# 结论: 通过 word_ids() 方法可以获取 token 对应的 word 索引

[None, 0, 1, 2, 2, 2, 2, None]

In [21]:
def process_function(examples):
    """通过 word_ids 将 token 与 word 对应，进而为每个 token 分配 word 的标签"""
    # examples 相当于一个 batch 的数据
    tokenized_examples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        # [None, 0, 1, 2, 2, 2, 2, None]
        for w in word_ids:
            if w is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[w])
        labels.append(label_ids)
    # 处理好（batch 中每个句子）每个 token 的标签后将 labels 作为新字段
    tokenized_examples["labels"] = labels
    return tokenized_examples

In [22]:
tokenized_dataset = dataset.map(process_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [23]:
tokenized_dataset["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  3862,
  7157,
  3683,
  6612,
  1765,
  4157,
  1762,
  1336,
  7305,
  680,
  7032,
  7305,
  722,
  7313,
  4638,
  3862,
  1818,
  511,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}

### 4. 创建模型

In [25]:
# 注意: 对于非二分类任务，num_labels 需要设置为标签（类别）的数量
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", cache_dir="./model", num_labels=len(label_lst))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model.config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
 

In [27]:
model.config.num_labels

7

### 5. 创建评估函数

In [ ]:
seqeval = evaluate.load("seqeval", cache_dir="./metric")
seqeval # 可以看一下输出示例

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [ ]:
def eval_metric(pred):
    """评估函数: 计算 f1 分数"""
    # pred 是一个包含两个元素的元组
    predictions, labels = pred  # (batch_size, sequence_length, num_labels) (batch_size, sequence_length)  
    predictions = np.argmax(predictions, axis=-1)

    # 将 id 转换为 string 类型的标签 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
    # 外层循环遍历每个句子 内层循环遍历每个 token
    true_predictions = [
        [label_lst[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    true_labels = [
        [label_lst[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    ret = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    # 可以返回多个键值对 训练日志每个 epoch 也会显示多个指标输出
    return {"f1": ret["overall_f1"]}    

### 6. 配置训练参数和训练器

In [37]:
args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=10,
    num_train_epochs=2
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
    compute_metrics=eval_metric
)

### 7. 模型训练和评估

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.031900,0.018619,0.949644
2,0.007400,0.016751,0.957211


TrainOutput(global_step=2610, training_loss=0.02527966531657282, metrics={'train_runtime': 1882.2799, 'train_samples_per_second': 22.17, 'train_steps_per_second': 1.387, 'total_flos': 2193181699685394.0, 'train_loss': 0.02527966531657282, 'epoch': 2.0})

In [40]:
trainer.evaluate(eval_dataset=tokenized_dataset["test"])

{'eval_loss': 0.021405352279543877,
 'eval_f1': 0.9485446637671462,
 'eval_runtime': 72.8093,
 'eval_samples_per_second': 63.687,
 'eval_steps_per_second': 1.992,
 'epoch': 2.0}

### 8. 模型预测

In [41]:
# 使用 pipeline 进行预测，指定一下 id2label，把标签 id 映射为 string 名称
model.config.id2label = {i: label for i, label in enumerate(label_lst)}
model.config

BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch

In [ ]:
# aggregation_strategy="simple" 用于将连续的相同实体 token 合并为一个实体
pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cuda:0


In [43]:
ret = pipe("李华在西安当牛马")
ret

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'PER',
  'score': np.float32(0.9994542),
  'word': '李 华',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': np.float32(0.99866533),
  'word': '西 安',
  'start': 3,
  'end': 5}]

In [45]:
txt = "李华在西安当牛马，她的对象张三在深圳上学。"
ret = pipe(txt)
ret

[{'entity_group': 'PER',
  'score': np.float32(0.9995997),
  'word': '李 华',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': np.float32(0.99891275),
  'word': '西 安',
  'start': 3,
  'end': 5},
 {'entity_group': 'PER',
  'score': np.float32(0.99949884),
  'word': '张 三',
  'start': 13,
  'end': 15},
 {'entity_group': 'LOC',
  'score': np.float32(0.9991808),
  'word': '深 圳',
  'start': 16,
  'end': 18}]

In [46]:
ret_dict = {}
for item in ret:
    if item["entity_group"] not in ret_dict:
        ret_dict[item["entity_group"]] = []
    ret_dict[item["entity_group"]].append(txt[item["start"]:item["end"]])

ret_dict

{'PER': ['李华', '张三'], 'LOC': ['西安', '深圳']}